In [ ]:
from keras.applications.resnet import ResNet50

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from keras.datasets import cifar10
from keras.layers import Input,Dense,Flatten,Dropout,Conv2DTranspose
from keras.models import Model, Sequential

from keras.utils import to_categorical


In [ ]:
(x_train, y_train), (x_valid, y_valid) = cifar10.load_data()
print(x_train.shape,y_train.shape)

(50000, 32, 32, 3) (50000, 1)


In [ ]:
y_train = y_train.reshape(-1,)

In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
classes5 = np.where(y_train < 5)

In [ ]:
x = x_train[classes5]

In [ ]:
y = y_train[classes5]

In [ ]:
print(x.shape,y.shape)

(25000, 32, 32, 3) (25000,)


In [ ]:
x = x.reshape(x.shape[0],32,32,3)
y = to_categorical(y,num_classes=5)

In [ ]:
print(x.shape,y.shape)

(25000, 32, 32, 3) (25000, 5)


In [ ]:
x = x.astype('float')/255.0

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,shuffle=True)
x_train.shape

(20000, 32, 32, 3)

In [ ]:
x_test,x_valid,y_test,y_valid = train_test_split(x_test,y_test ,test_size= 0.5)
x_test.shape

(2500, 32, 32, 3)

In [ ]:
print(x_train.shape,x_test.shape,x_valid.shape)

(20000, 32, 32, 3) (2500, 32, 32, 3) (2500, 32, 32, 3)


In [ ]:
input_tensor = Input(shape=(32,32,3))
inp = Conv2DTranspose(3,(2,2),strides=(2,2))(input_tensor);
inp = Conv2DTranspose(3,(2,2),strides=(2,2))(inp);

In [ ]:
model = ResNet50( weights = 'imagenet' , include_top = False)
model = model(inp)

In [ ]:
model.summary()


In [ ]:
flat= Flatten(name='flatten')(model)
out = Dense(1024,activation= 'relu', name = 'fc_1')(flat)
out = Dense(512,activation = 'relu',name = 'fc_2')(out)
out = Dense(5,activation = 'softmax' ,name = 'fc_3')(out)
custom_resnet_model = Model(inputs=input_tensor,outputs= out)
custom_resnet_model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_transpose_26 (Conv2DT (None, 64, 64, 3)         39        
_________________________________________________________________
conv2d_transpose_27 (Conv2DT (None, 128, 128, 3)       39        
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
fc_1 (Dense)                 (None, 1024)              33555456  
_________________________________________________________________
fc_2 (Dense)                 (None, 512)              

In [ ]:
for layer in custom_resnet_model.layers[5:-5]:
	layer.trainable = False

In [ ]:
for layer in custom_resnet_model.layers:
  print(layer.name,":",layer.trainable)
custom_resnet_model.summary()

input_31 : True
conv2d_transpose_26 : True
conv2d_transpose_27 : True
resnet50 : True
flatten : True
fc_1 : True
fc_2 : True
fc_3 : True
Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_transpose_26 (Conv2DT (None, 64, 64, 3)         39        
_________________________________________________________________
conv2d_transpose_27 (Conv2DT (None, 128, 128, 3)       39        
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
fc_1 (Dense)                 (None, 1024)        

In [ ]:
custom_resnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
custom_resnet_model.fit(x_train, y_train, batch_size=32, epochs=30, verbose=1, validation_data=(x_valid, y_valid))

Epoch 1/30
625/625 [==============================] - 85s 136ms/step - loss: 1.0384 - accuracy: 0.6745 - val_loss: 4.1887 - val_accuracy: 0.2004
Epoch 2/30
625/625 [==============================] - 84s 134ms/step - loss: 0.5468 - accuracy: 0.8087 - val_loss: 1.6830 - val_accuracy: 0.3692
Epoch 3/30
625/625 [==============================] - 84s 134ms/step - loss: 0.4108 - accuracy: 0.8565 - val_loss: 0.4714 - val_accuracy: 0.8260
Epoch 4/30
625/625 [==============================] - 84s 134ms/step - loss: 0.3867 - accuracy: 0.8680 - val_loss: 0.6942 - val_accuracy: 0.7680
Epoch 5/30
625/625 [==============================] - 84s 134ms/step - loss: 0.4465 - accuracy: 0.8493 - val_loss: 0.7792 - val_accuracy: 0.7108
Epoch 6/30
625/625 [==============================] - 84s 134ms/step - loss: 0.3507 - accuracy: 0.8794 - val_loss: 0.5050 - val_accuracy: 0.8252
Epoch 7/30
625/625 [==============================] - 84s 134ms/step - loss: 0.2154 - accuracy: 0.9252 - val_loss: 0.4439 - val_ac

In [ ]:
custom_resnet_model.evaluate(x_test,y_test,batch_size=1,verbose = 1)

2500/2500 [==============================] - 21s 8ms/step - loss: 0.7280 - accuracy: 0.8580


[0.7279995679855347, 0.8579999804496765]